In [ ]:
!pip install numpy pandas tqdm

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [3]:
n_teams = 10
n_events = n_teams // 2
n_slots = n_events

In [4]:
class Matchup:
    def __init__(self, event: int):
        self.event = event
        self.team_a = -1
        self.team_b = -1
    
    @property
    def slot_free(self):
        return self.team_a == -1 or self.team_b == -1

    def add_team(self, team_n):
        if not self.slot_free:
            raise AssertionError(f"No Free spot for event {self.event}! Matching team {self.team_a} and team {self.team_b}")
        if self.team_a == -1:
            self.team_a = team_n
        else:
            self.team_b = team_n

    def has_team(self, team_n):
        return team_n in [self.team_a, self.team_b]

    def __str__(self):
        return f"Event {self.event}, match: {self.team_a} \t {self.team_b}"

class Team:
    N_EVENTS = n_events
    def __init__(self, team_number:int):
        self.team_number = team_number
        self.matched_teams = []
        self.schedule = []

    def gen_event_p(self, matchups: 'list[Matchup]'):
        output = np.ones(self.N_EVENTS)
        output[self.schedule] = 0
        for m in matchups:
            if not m.slot_free or any([m.has_team(opponent) for opponent in self.matched_teams]):
                output[m.event] = 0
        return output / np.sum(output)

    def partitipate_in_event(self, i):
        self.schedule.append(i)
    
    def select_next_event(self, matchups: 'list[Matchup]'):
        return np.random.choice(range(self.N_EVENTS), p=self.gen_event_p(matchups))

    def validate_schedule(self):
        try:
            assert len(set(self.schedule)) == len(self.schedule)
        except AssertionError as e:
            print("ERROR! participated in an event twice!")
            print(self.schedule)
            raise e
        try:
            assert len(set(self.matched_teams)) == len(self.matched_teams)
        except AssertionError as e:
            print("ERROR! competed against the same team twice!")
            print(self.matched_teams)
            raise e


ts = time.time()
for attempt in tqdm(range(10000000)):
    success = False
    teams = [Team(t)for t in range(n_teams)]
    try:
        for timeslot in range(n_slots):
            # prepare the timeslot
            matchups = [Matchup(i) for i in range(n_events)]
            # iterate over teams
            for i, team in enumerate(teams):
                #iterate over opponents to see which events are free
                team.partitipate_in_event(team.select_next_event(matchups))
                matchups[team.schedule[-1]].add_team(team.team_number)
            for m in matchups:
                teams[m.team_a].matched_teams.append(m.team_b)
                teams[m.team_b].matched_teams.append(m.team_a)
            try:
                for t in teams:
                    t.validate_schedule()
            except AssertionError:
                for t in teams:
                    t.matched_teams = t.matched_teams[:-1]
        success = True
        print(f"Took {attempt+1} tries, but I got it!")
        break
    except ValueError:
        pass
if success:
    team_schedule = np.vstack([t.schedule for t in teams])
    print(team_schedule)
    print(f"Took {time.time() - ts}s")
else:
    print("no success...")

  0%|          | 0/10000000 [00:00<?, ?it/s]/tmp/ipykernel_10294/1717096321.py:38: RuntimeWarning: invalid value encountered in divide
  return output / np.sum(output)
  0%|          | 774/10000000 [00:01<5:18:48, 522.75it/s]

Took 775 tries, but I got it!
[[4 0 3 2 1]
 [4 3 0 1 2]
 [1 0 2 4 3]
 [0 4 1 3 2]
 [2 3 1 4 0]
 [3 4 2 0 1]
 [0 2 4 1 3]
 [2 1 3 0 4]
 [3 1 4 2 0]
 [1 2 0 3 4]]
Took 1.4867660999298096s


In [5]:
teams = 'ABCDEFGHIJKLMNOPGRSTUVWXYZ'  # 26 teams is enormous... so should be enough...
for i,row in enumerate(team_schedule):
    print(teams[i], row+1)

A [5 1 4 3 2]
B [5 4 1 2 3]
C [2 1 3 5 4]
D [1 5 2 4 3]
E [3 4 2 5 1]
F [4 5 3 1 2]
G [1 3 5 2 4]
H [3 2 4 1 5]
I [4 2 5 3 1]
J [2 3 1 4 5]
